In [2]:
import sympy as sp
sp.init_printing(use_unicode=True)

t  = sp.symbols('t')
q1, q2, q3 = sp.Function('q1'), sp.Function('q2'), sp.Function('q3')
s1, s2, s3 = sp.sin(q1(t)), sp.sin(q2(t)), sp.sin(q3(t))
c1, c2, c3 = sp.cos(q1(t)), sp.cos(q2(t)), sp.cos(q3(t))
e1, e2, e3 = sp.Matrix([1, 0, 0]), sp.Matrix([0, 1, 0]), sp.Matrix([0, 0, 1])

# Rotations
# See "Dynamic modeling and characteristics analysis of lateral-pendulum unicycle robot" by Daud et al
# (The columns of the rotation matrix are the wheel frame unit vectors expressed contact point frame)

# Rotation from wheel frame to contact point frame
R1 = sp.Matrix([[1, 0., 0.],
                [0., c1, -s1],
                [0., s1, c1]])

# Rotation from body frame to wheel frame
R2 = sp.Matrix([[c2, 0., s2],
                [0., 1., 0.],
                [-s2, 0., c2]])

# Rotation from contact point frame to inertial frame
R3 = sp.Matrix([[c3, -s3, 0.],
                [s3, c3, 0.],
                [0., 0., 1.]])

#R_BI = R2.T * R1.T * R3.T
R_IB = R3 * R1 * R2 # Rotation of B frame expressed in I frame

dR_IB = sp.diff(R_IB, t)

"""
The formula below expresses the angular velocity "omega" relative to the inertia frame in the body frame.
See:
- Page 78, Lynch - "Modern Robotics"
- Page 3, Kim - "Lie Group Formulation of Articulated Rigid Body Dynamics"
"""
omega_B_mat = R_IB.T * dR_IB

def so3toVec(R):
    """
    Converts an so(3) matrix to vec using the Schur complement
    Source: Modern Robotics - https://github.com/NxRLab/ModernRobotics
    """
    return sp.Matrix([R[2, 1], R[0, 2], R[1, 0]])

omega_B_vec = sp.simplify(so3toVec(omega_B_mat))

col1 = omega_B_vec.subs([(sp.Derivative(q1(t), t), 1), (sp.Derivative(q2(t), t), 0), (sp.Derivative(q3(t), t), 0)])
col2 = omega_B_vec.subs([(sp.Derivative(q1(t), t), 0), (sp.Derivative(q2(t), t), 1), (sp.Derivative(q3(t), t), 0)])
col3 = omega_B_vec.subs([(sp.Derivative(q1(t), t), 0), (sp.Derivative(q2(t), t), 0), (sp.Derivative(q3(t), t), 1)])
J_euler2omega = sp.simplify(col1.row_join(col2).row_join(col3))
J_omega2euler_liegroup = sp.simplify(J_euler2omega.inv())

# Expression from the paper
J_omega2euler_wrong=  (e1.T * R2).col_join(e2.T).col_join(e3.T*R1*R2)

# I again did the derivation by hand...
# Note:
# - dq2 points along the rotation axis between the b-frame and w-frame (second axis), so it is already in the b frame
# - dq1 points along the rotation axis between the w-frame and cp-frame (first axis), so it is transformed from w-frame to b-frame
# - dq3 points along the rotation axis between the cp-frame and inertial frame (third axis), so it is transformed from cp-frame to b-frame
J_euler2omega_new =  sp.Matrix.hstack(R2.T*e1, e2, R2.T*R1.T*e3)
J_omega2euler_new = sp.simplify(J_euler2omega_new.inv()) # <- HERE IS THE MISTAKE, I DIDN'T INVERT THE MATRIX BUT TRANSPOSED IT

def make_rot_readable(R):
    return R.subs([(c1, 'c1'), (c2, 'c2'), (c3, 'c3'), (s1, 's1'), (s2, 's2'), (s3, 's3'),
                   (sp.sin(2*q1(t)), 2*s1*c1), (sp.sin(2*q2(t)), 2*s2*c2), (sp.sin(2*q3(t)), 2*s3*c3)])

J_omega2euler_wrong= make_rot_readable(J_omega2euler_wrong)
J_omega2euler_liegroup = make_rot_readable(J_omega2euler_liegroup)
J_omega2euler_new = make_rot_readable(J_omega2euler_new)

print(f'J_omega2euler_wrong: \n {J_omega2euler_wrong}')
print(f'J_omega2euler_liegroup: \n {J_omega2euler_liegroup}')
print(f'J_omega2euler_new: \n {J_omega2euler_new}')


J_omega2euler_wrong: 
 Matrix([[c2, 0, s2], [0, 1, 0], [-c1*s2, 1.0*s1, c1*c2]])
J_omega2euler_liegroup: 
 Matrix([[1.0*c2, 0, 1.0*s2], [1.0*s2*tan(q1(t)), 1, -1.0*c2*tan(q1(t))], [-1.0*s2/c1, 0, 1.0*c2/c1]])
J_omega2euler_new: 
 Matrix([[c2, 0, s2], [1.0*s2*tan(q1(t)), 1, -1.0*c2*tan(q1(t))], [-s2/c1, 0, c2/c1]])
